
# Aggregate Metrics by UMLS CUI and Join to Crosswalk

This notebook produces a single table of metrics per **UMLS_CUI** by aggregating MarketScan (claims), FAERS (cases), and MPRINT (publication counts), then **joining onto `final_joined.csv`**.

**Outputs**
- `drug_metrics_by_cui.csv` (in the same `cross_quartz` folder as your maps)
- On-screen QC summaries and previews

**Notes**
- Auto-detects sensible columns (FAERS case IDs, MarketScan patient IDs).
- MPRINT can be sourced from a publications table with a CUI column or a simple list of CUIs (`mprint_druglist.txt`); both are supported.



## 1) Configure paths
Defaults use your OneDrive locations. Adjust if needed.


In [1]:

from pathlib import Path

# Root dir where crosswalks live
MAP_DIR = Path("/Users/rahurkar.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/FAERS/drug_id_platform/cross_quartz")

# Inputs expected from the prior step (written next to the crosswalks)
MS_WITH_CUI     = MAP_DIR / "ms_with_cui.csv"
FAERS_WITH_CUI  = MAP_DIR / "faers_with_cui.csv"

# Crosswalks / helpers
FINAL_JOINED    = MAP_DIR / "final_joined.csv"
MPRINT_STD      = MAP_DIR / "mprint_standardized.csv"   # optional fallback if it includes per-publication rows
MPRINT_LIST_TXT = MAP_DIR / "mprint_druglist.txt"       # optional: a simple list with column 'cui' (each row = a pub)

# --- Local fallbacks for sandbox/demo runs (ignore on your machine) ---
if not MS_WITH_CUI.exists():
    MS_WITH_CUI = Path("/mnt/data/ms_with_cui.csv")
if not FAERS_WITH_CUI.exists():
    FAERS_WITH_CUI = Path("/mnt/data/faers_with_cui.csv")
if not FINAL_JOINED.exists():
    FINAL_JOINED = Path("/mnt/data/final_joined.csv")
if not MPRINT_STD.exists():
    MPRINT_STD = Path("/mnt/data/mprint_standardized.csv")
if not MPRINT_LIST_TXT.exists():
    MPRINT_LIST_TXT = Path("/mnt/data/mprint_druglist.txt")

MAP_DIR, MS_WITH_CUI, FAERS_WITH_CUI, FINAL_JOINED, MPRINT_STD, MPRINT_LIST_TXT


(PosixPath('/Users/rahurkar.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/FAERS/drug_id_platform/cross_quartz'),
 PosixPath('/Users/rahurkar.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/FAERS/drug_id_platform/cross_quartz/ms_with_cui.csv'),
 PosixPath('/Users/rahurkar.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/FAERS/drug_id_platform/cross_quartz/faers_with_cui.csv'),
 PosixPath('/Users/rahurkar.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/FAERS/drug_id_platform/cross_quartz/final_joined.csv'),
 PosixPath('/Users/rahurkar.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/FAERS/drug_id_platform/cross_quartz/mprint_standardized.csv'),
 PosixPath('/mnt/data/mprint_druglist.txt'))


## 2) Helpers
- Column auto-detection for **MarketScan patient id** and **FAERS case id**
- Safe normalization
- Simple QC summary builder


In [2]:

import pandas as pd

def normalize(s: pd.Series) -> pd.Series:
    return (
        s.fillna("")
         .astype(str)
         .str.lower()
         .str.strip()
         .str.replace(r"\s+", " ", regex=True)
    )

# Candidates for auto-detection
MS_PATIENT_COL_CANDIDATES = [
    "enrolid", "enrolleeid", "patient_id", "person_id", "memberid", "patid", "personid"
]
FAERS_CASEID_COL_CANDIDATES = [
    "caseid", "primaryid", "safetyreportid", "isr"
]
CUI_COL_CANDIDATES = ["UMLS_CUI", "CUI", "cui"]

def find_first_present(df: pd.DataFrame, candidates: list[str]) -> str | None:
    lower_map = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in lower_map:
            return lower_map[cand.lower()]
    return None

def summarize_qc_counts(df: pd.DataFrame, label: str, cui_col="UMLS_CUI") -> pd.DataFrame:
    total = len(df)
    with_cui = df[cui_col].notna().sum() if cui_col in df.columns else 0
    return pd.DataFrame([{
        "source": label,
        "rows": total,
        "with_cui": with_cui,
        "without_cui": total - with_cui,
        "cui_rate": round(with_cui / total, 4) if total else 0.0
    }])



## 3) Load inputs & inspect columns


In [3]:

# Load
ms_with   = pd.read_csv(MS_WITH_CUI, dtype=str, low_memory=False)
faers_with= pd.read_csv(FAERS_WITH_CUI, dtype=str, low_memory=False)
key       = pd.read_csv(FINAL_JOINED, dtype={"UMLS_CUI":"string"}, low_memory=False)

print("MS with CUI columns   :", list(ms_with.columns))
print("FAERS with CUI columns:", list(faers_with.columns))
print("final_joined columns  :", list(key.columns))

# Harmonize CUI column name casing
def coerce_cui_col(df: pd.DataFrame) -> pd.DataFrame:
    for c in CUI_COL_CANDIDATES:
        if c in df.columns:
            if c != "UMLS_CUI":
                df = df.rename(columns={c: "UMLS_CUI"})
            break
    return df

ms_with    = coerce_cui_col(ms_with)
faers_with = coerce_cui_col(faers_with)
key        = coerce_cui_col(key)

# Show a quick QC of input CUI coverage
pd.concat([
    summarize_qc_counts(ms_with, "MarketScan"),
    summarize_qc_counts(faers_with, "FAERS"),
], ignore_index=True)


MS with CUI columns   : ['drug_ms', 'clean_drug', 'base_drug', 'UMLS_CUI', 'Preferred_Term', 'Preferred_TTY']
FAERS with CUI columns: ['fda_drug', 'clean_drug', 'base_drug', 'UMLS_CUI', 'Preferred_Term', 'Preferred_TTY']
final_joined columns  : ['UMLS_CUI', 'Preferred_Term', 'MS_flag', 'FAERS_flag', 'MPRINT_flag']


,source,rows,with_cui,without_cui,cui_rate
0,MarketScan,2584,2145,439,0.8301
1,FAERS,21816,13870,7946,0.6358



## 4) Aggregate by CUI
- MarketScan: **ms_rx_count** (rows with a CUI) and **ms_unique_patients** (if a patient column is found).
- FAERS: **faers_case_count** (unique case IDs by CUI). If no case ID column is found, falls back to row count.
- MPRINT: **pub_count** using either:
  - A publications file with a CUI column (counts rows per CUI), or
  - A simple list file (`mprint_druglist.txt`) with a column `cui` (each row represents a publication).


In [4]:

# --- MarketScan aggregation ---
ms_pat_col = find_first_present(ms_with, MS_PATIENT_COL_CANDIDATES)

ms_claims = (
    ms_with[ms_with["UMLS_CUI"].notna()]
      .groupby("UMLS_CUI", dropna=True)
      .size()
      .rename("ms_rx_count")
      .reset_index()
)

if ms_pat_col:
    ms_pat = (
        ms_with[ms_with["UMLS_CUI"].notna()]
          .groupby("UMLS_CUI")[ms_pat_col]
          .nunique()
          .rename("ms_unique_patients")
          .reset_index()
    )
    ms_agg = ms_claims.merge(ms_pat, on="UMLS_CUI", how="left")
else:
    ms_agg = ms_claims.assign(ms_unique_patients=pd.NA)

# --- FAERS aggregation ---
faers_case_col = find_first_present(faers_with, FAERS_CASEID_COL_CANDIDATES)

if faers_case_col:
    faers_agg = (
        faers_with[faers_with["UMLS_CUI"].notna()]
          .groupby("UMLS_CUI")[faers_case_col]
          .nunique()
          .rename("faers_case_count")
          .reset_index()
    )
else:
    # fallback if we can't find a case id column: row counts
    faers_agg = (
        faers_with[faers_with["UMLS_CUI"].notna()]
          .groupby("UMLS_CUI", dropna=True)
          .size()
          .rename("faers_case_count")
          .reset_index()
    )

# --- MPRINT aggregation ---
pub_agg = None

# 1) Try a table with a recognizable CUI column
for candidate_path in [MPRINT_STD, MPRINT_LIST_TXT]:
    if candidate_path.exists():
        try:
            if candidate_path.suffix.lower() in [".txt", ".tsv"]:
                tmp = pd.read_csv(candidate_path, sep="\t", dtype=str, low_memory=False)
            else:
                tmp = pd.read_csv(candidate_path, dtype=str, low_memory=False)
        except Exception:
            continue

        # find a CUI column and count rows per CUI
        cui_col = find_first_present(tmp, CUI_COL_CANDIDATES)
        if cui_col:
            tmp = tmp.rename(columns={cui_col: "UMLS_CUI"})
            pub_agg = (
                tmp[tmp["UMLS_CUI"].notna()]
                  .groupby("UMLS_CUI", dropna=True)
                  .size()
                  .rename("pub_count")
                  .reset_index()
            )
            break

# If none found, create an empty frame
if pub_agg is None:
    pub_agg = pd.DataFrame({"UMLS_CUI": pd.Series(dtype="string"), "pub_count": pd.Series(dtype="Int64")})

ms_pat_col, faers_case_col, ms_agg.head(), faers_agg.head(), pub_agg.head()


(None,
 None,
    UMLS_CUI  ms_rx_count ms_unique_patients
 0  C0000294            1               <NA>
 1  C0000378            1               <NA>
 2  C0000477            1               <NA>
 3  C0000608            1               <NA>
 4  C0000618            1               <NA>,
    UMLS_CUI  faers_case_count
 0  C0000266                 1
 1  C0000294                13
 2  C0000378                 1
 3  C0000477                 1
 4  C0000503                 1,
 Empty DataFrame
 Columns: [UMLS_CUI, pub_count]
 Index: [])


## 5) Join aggregates onto `final_joined.csv`


In [5]:

# Ensure key has the columns we expect
expected_key_cols = ["UMLS_CUI","Preferred_Term","MS_flag","FAERS_flag","MPRINT_flag"]
missing = [c for c in expected_key_cols if c not in key.columns]
if missing:
    print("[warn] final_joined.csv missing columns:", missing)

metrics = (
    key[["UMLS_CUI","Preferred_Term","MS_flag","FAERS_flag","MPRINT_flag"]].copy()
      .merge(ms_agg,   on="UMLS_CUI", how="left")
      .merge(faers_agg,on="UMLS_CUI", how="left")
      .merge(pub_agg,  on="UMLS_CUI", how="left")
)

# Fill numeric columns
for col in ["ms_rx_count","ms_unique_patients","faers_case_count","pub_count"]:
    if col in metrics.columns:
        if col == "ms_unique_patients":
            # may be entirely NA if patient column missing
            try:
                metrics[col] = metrics[col].astype("Int64")
            except Exception:
                pass
        else:
            metrics[col] = metrics[col].fillna(0)
            try:
                metrics[col] = metrics[col].astype("int64")
            except Exception:
                metrics[col] = metrics[col].astype("Int64")

metrics.head(10)


,UMLS_CUI,Preferred_Term,MS_flag,FAERS_flag,MPRINT_flag,ms_rx_count,ms_unique_patients,faers_case_count,pub_count
0,C3498054,5-methyltetrahydrofolic acid,1,0,0,2,<NA>,0,0
1,C0724515,abacavir sulfate,1,1,0,1,<NA>,2,0
2,C0663655,abacavir,1,1,0,3,<NA>,34,0
3,C1619966,abatacept,1,1,0,1,<NA>,4,0
4,C3852841,abemaciclib,1,1,0,1,<NA>,1,0
5,C2607886,abiraterone acetate,1,1,0,1,<NA>,2,0
6,C2719424,abobotulinumtoxinA,1,1,0,1,<NA>,3,0
7,C5139788,abrocitinib,1,1,0,1,<NA>,1,0
8,C2727031,Acacia allergenic extract 0.01 GM/ML Injectabl...,1,0,0,1,<NA>,0,0
9,C0592789,acamprosate calcium,1,0,0,1,<NA>,0,0



## 6) Write output and basic QC


In [6]:

# Write CSV
out_fn = MAP_DIR / "drug_metrics_by_cui.csv"
out_fn.parent.mkdir(parents=True, exist_ok=True)
metrics.to_csv(out_fn, index=False)

# QC
print("Wrote:", out_fn)
print("\nNon-zero counts summary:")
summary_cols = [c for c in ["ms_rx_count","ms_unique_patients","faers_case_count","pub_count"] if c in metrics.columns]
display(metrics[summary_cols].gt(0).sum().to_frame("# CUIs with >0").T)

print("\nTop 10 by FAERS cases:")
if "faers_case_count" in metrics.columns:
    display(metrics.sort_values("faers_case_count", ascending=False).head(10)[["UMLS_CUI","Preferred_Term","faers_case_count"]])

print("\nTop 10 by MarketScan claims:")
if "ms_rx_count" in metrics.columns:
    display(metrics.sort_values("ms_rx_count", ascending=False).head(10)[["UMLS_CUI","Preferred_Term","ms_rx_count"]])

print("\nTop 10 by publications:")
if "pub_count" in metrics.columns:
    display(metrics.sort_values("pub_count", ascending=False).head(10)[["UMLS_CUI","Preferred_Term","pub_count"]])

out_fn


Wrote: /Users/rahurkar.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/FAERS/drug_id_platform/cross_quartz/drug_metrics_by_cui.csv

Non-zero counts summary:


,ms_rx_count,ms_unique_patients,faers_case_count,pub_count
# CUIs with >0,1787,0,5261,0



Top 10 by FAERS cases:


,UMLS_CUI,Preferred_Term,faers_case_count
85,C0308321,Amoxi Drop,143
948,C0377265,levetiracetam,83
2281,C0013089,doxorubicin,70
3444,C0042679,vincristine,67
479,C0011777,dexamethasone,60
1653,C0040341,tobramycin,60
1724,C0042313,vancomycin,59
636,C0015133,etoposide,58
260,C0006686,calcium chloride,54
922,C0064636,lamotrigine,53



Top 10 by MarketScan claims:


,UMLS_CUI,Preferred_Term,ms_rx_count
257,C0006675,calcium,14
811,C0020261,hydrochlorothiazide,11
14,C0718194,Aceta,11
79,C0051696,amlodipine,8
226,C0006246,brompheniramine,8
960,C0023660,lidocaine,7
13,C0000970,acetaminophen,7
116,C0003968,ascorbic acid,7
117,C4688975,Ascor,7
122,C0004057,aspirin,7



Top 10 by publications:


,UMLS_CUI,Preferred_Term,pub_count
0,C3498054,5-methyltetrahydrofolic acid,0
3857,C0008221,clomethiazole,0
3855,C1245600,clobazam Oral Capsule,0
3854,C2345299,Clinimix 2.75/5,0
3853,C1126116,clindamycin 300 MG,0
3852,C1105819,Climara,0
3851,C4535082,Clenpiq,0
3850,C0008932,clenbuterol,0
3849,C0008929,clemastine,0
3848,C2724955,ClearLax,0


PosixPath('/Users/rahurkar.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/FAERS/drug_id_platform/cross_quartz/drug_metrics_by_cui.csv')